In [13]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [14]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [15]:
def conv_layer(input,channels_in,channels_out,name ="conv"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([5,5,channels_in,channels_out] ,stddev=0.1),name ="W")
        b = tf.Variable(tf.constant(0.1,shape= [channels_out]),name ="B")
        conv = tf.nn.conv2d(input,w,strides = [1,1,1,1],padding="SAME")
        act = tf.nn.relu(conv+b)
        tf.summary.histogram("weights",w)
        tf.summary.histogram("biases",b)
        tf.summary.histogram("activations",act)

        return act

In [16]:
def fc_layer(input,channels_in,channels_out,name ="fc"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([channels_in,channels_out],stddev=0.1),name ="W")
        b = tf.Variable(tf.constant(0.1,shape=[channels_out]),name="B")
        act = tf.nn.relu(tf.matmul(input,w) + b)

        return act

In [17]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32,shape=[None,784],name="x")
y = tf.placeholder(tf.float32,shape=[None,10],name="labels")
x_image = tf.reshape(x,[-1,28,28,1])
tf.summary.image('input',x_image,3)

conv1 = conv_layer(x_image,1,32,"conv1")
pool1 = tf.nn.max_pool(conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

conv2 = conv_layer(pool1,32,64,"conv2")
pool2 = tf.nn.max_pool(conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
flattened = tf.reshape(pool2,[-1,7*7*64])

fc1 = fc_layer(flattened,7*7*64,1024,"fc1")
logits = fc_layer(fc1,1024,10,"fc2")

with tf.name_scope("cross_entropy"):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y))
    tf.summary.scalar('cross_entropy',cross_entropy)
    
with tf.name_scope("train"):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(logits,1),tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    tf.summary.scalar("accuracy",accuracy)
    
with tf.Session() as sess:
    merged_summary =  tf.summary.merge_all()
    writer = tf.summary.FileWriter("/tmp/mnist_demo/2")
    writer.add_graph(sess.graph)
    sess.run(tf.global_variables_initializer())

    for i in range(20000):
        batch = mnist.train.next_batch(100)

        if i % 500 == 0:
            [train_accuracy] = sess.run([accuracy],feed_dict =  {x:batch[0] , y:batch[1]})
            summary = sess.run(merged_summary , feed_dict={x:batch[0],y:batch[1]})
            print("step %d ,training accuracy %g" % (i,train_accuracy))
            writer.add_summary(summary,i)

        sess.run(train_step,feed_dict = {x:batch[0],y:batch[1]})

step 0 ,training accuracy 0.15
step 500 ,training accuracy 0.1
step 1000 ,training accuracy 0.09
step 1500 ,training accuracy 0.08
step 2000 ,training accuracy 0.07
step 2500 ,training accuracy 0.06
step 3000 ,training accuracy 0.13
step 3500 ,training accuracy 0.02
step 4000 ,training accuracy 0.28
step 4500 ,training accuracy 0.35
step 5000 ,training accuracy 0.39
step 5500 ,training accuracy 0.5
step 6000 ,training accuracy 0.57
step 6500 ,training accuracy 0.62
step 7000 ,training accuracy 0.65
step 7500 ,training accuracy 0.56
step 8000 ,training accuracy 0.59
step 8500 ,training accuracy 0.52
step 9000 ,training accuracy 0.55
step 9500 ,training accuracy 0.61
step 10000 ,training accuracy 0.57
step 10500 ,training accuracy 0.59
step 11000 ,training accuracy 0.58
step 11500 ,training accuracy 0.74
step 12000 ,training accuracy 0.66
step 12500 ,training accuracy 0.63
step 13000 ,training accuracy 0.6
step 13500 ,training accuracy 0.59
step 14000 ,training accuracy 0.6
step 14500 ,t

Launch tensorboard passing --logdir <path used in FileWriter> :
tensorboard --logdir /tmp/mnist_demo/2